This is your private sketch-notebook. Do whatever you want with it. When code is ready to be developed, please just copy and paste it to the final notebook. 

In [1]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  
from PIL import Image
import glob
import numpy as np
import pandas as pd
from IPython.display import Image
from IPython.display import Image
from IPython.display import Image
from IPython.core.display import HTML 


Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# the folder path to the folder with all the fishies in there
folders_path = '../Data/Kaggle/train/*'

# creating a dataframe with the columns below
image_dict = {}

# going through all folders and saving file names in dataframe so that order is always correct
for foldername in glob.glob(folders_path):
    for image_name in glob.glob(foldername + '/*.jpg'):
        # getting image name and fish type and putting it in a dictionary
        fish_type = foldername[-3:]
        image_dict[image_name] = fish_type

image_df = pd.DataFrame(image_dict.items(), columns = ['image_name', 'fish_type']).sample(frac=1) # random sampling
dummy_df = pd.get_dummies(image_df, columns=['fish_type']) # dummify data

dummy_df.head()

,image_name,fish_type_ALB,fish_type_ARK,fish_type_BET,fish_type_DOL,fish_type_HER,fish_type_LAG,fish_type_NoF,fish_type_YFT
2035,../Data/Kaggle/train/YFT/img_05451.jpg,0,0,0,0,0,0,0,1
532,../Data/Kaggle/train/SHARK/img_07056.jpg,0,1,0,0,0,0,0,0
3515,../Data/Kaggle/train/YFT/img_01064.jpg,0,0,0,0,0,0,0,1
3141,../Data/Kaggle/train/NoF/img_00849.jpg,0,0,0,0,0,0,1,0
3074,../Data/Kaggle/train/DOL/img_07212.jpg,0,0,0,1,0,0,0,0


In [4]:
# Smaller dataset for testing purposes!
dummy_df = dummy_df.iloc[0:101,:]

In [5]:
Image(url= "https://kaggle2.blob.core.windows.net/competitions/kaggle/5568/media/species-ref-key.jpg")

In [6]:
# Count of the image types
image_df.groupby('fish_type').count().sort_values('image_name')

,image_name
fish_type,
LAG,67
DOL,117
ARK,176
BET,200
HER,299
NoF,465
YFT,734
ALB,1719


In [ ]:
#test cropping

In [7]:
count = 0 # count, to test the algorithm with 100 pictures
images = []

for filename in dummy_df.image_name:
    img = cv2.imread(filename) # Read file    
    #crop algorithm
    img = cv2.resize(img, (224, 224)) # Resize image to 224 x 224 pixels (dont feel obliged to use this)
    img = img.astype(np.float32) # Change to 32 bit floats (good for neural networks, other floats not as much making sense)

    # "Normalize" images
    for rgb_value in [0,1,2]:
        img[:, :, rgb_value] = img[:, :, rgb_value] / 255.0 # 255 is the RGB maximum
        img[:, :, rgb_value] -= numpy.mean(img[:, :, rgb_value])

    img = img.transpose((2,0,1))

    images.append(img)

# X and y value and labels
x = np.asarray(images)
y = dummy_df.iloc[:,1:9].as_matrix()
labels = dummy_df.image_name.apply(lambda path: path[-13:])

# table, just to be sure that everything goes well
dummy_df['image_bytes'] = images
dummy_df.head()

,image_name,fish_type_ALB,fish_type_ARK,fish_type_BET,fish_type_DOL,fish_type_HER,fish_type_LAG,fish_type_NoF,fish_type_YFT,image_bytes
2035,../Data/Kaggle/train/YFT/img_05451.jpg,0,0,0,0,0,0,0,1,"[[[0.261447, 0.257526, 0.253604, 0.245761, 0.2..."
532,../Data/Kaggle/train/SHARK/img_07056.jpg,0,1,0,0,0,0,0,0,"[[[-0.224212, -0.216369, -0.224212, -0.22029, ..."
3515,../Data/Kaggle/train/YFT/img_01064.jpg,0,0,0,0,0,0,0,1,"[[[0.0475199, 0.118108, 0.12203, 0.157324, 0.2..."
3141,../Data/Kaggle/train/NoF/img_00849.jpg,0,0,0,0,0,0,1,0,"[[[0.612205, 0.60044, 0.604361, 0.612205, 0.58..."
3074,../Data/Kaggle/train/DOL/img_07212.jpg,0,0,0,1,0,0,0,0,"[[[0.562017, 0.550252, 0.554173, 0.562017, 0.5..."


In [8]:
split_no = 90

x_train = x[:split_no]
y_train = y[:split_no]
labels_train = labels[:split_no]

x_test = x[split_no:]
y_test = y[split_no:]
labels_test = labels[split_no:]

In [9]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 224, 224), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
# Compile model
epochs = 1 # dont leave it like this, only for testing!
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 224, 224)  896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 32, 224, 224)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 224, 224)  9248        dropout_1[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 112, 112)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [10]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
classes = model.predict_proba(x_test, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print("Accuracy: %.2f%%" % (scores[1]*100))
print ' '
proba_df

Train on 90 samples, validate on 11 samples
Epoch 1/1
11/11 [==============================] - 0s
Accuracy: 36.36%
 


fish_type,ALB,ARK,BET,DOL,HER,LAG,NoF,YFT
0,0.422745,0.069238,0.086199,0.035231,0.109440,0.022538,0.064539,0.190069
1,0.420992,0.070306,0.076523,0.054648,0.122532,0.040406,0.075733,0.138859
2,0.457679,0.065891,0.069663,0.051688,0.119705,0.038711,0.066283,0.130380
3,0.415433,0.069366,0.080799,0.050382,0.135239,0.031828,0.071499,0.145454
4,0.403516,0.073418,0.082888,0.052705,0.125024,0.041299,0.074577,0.146573
5,0.235728,0.108895,0.123534,0.076905,0.120656,0.061831,0.101084,0.171367
6,0.436124,0.064616,0.068020,0.051332,0.118565,0.034690,0.073141,0.153511
7,0.397890,0.078257,0.081822,0.044309,0.135606,0.028858,0.061682,0.171575
8,0.325743,0.086648,0.098861,0.074454,0.127212,0.053997,0.086764,0.146321
9,0.285287,0.088994,0.098564,0.082846,0.122497,0.060059,0.097476,0.164277


In [11]:
proba_df['img_names'] = labels_test.values